# Multiple Choice Answer Type

This notebook demonstrates how to generate multiple choice questions with predefined answer options. Multiple choice questions are perfect for educational content, knowledge testing, and structured predictions with a fixed set of choices.

In [ ]:
%pip install lightningrod-ai

from IPython.display import clear_output
clear_output()

In [ ]:
import os
from datetime import datetime, timedelta
from lightningrod import (
    LightningRod,
    NewsSeedGenerator,
    QuestionAndLabelGenerator,
    QuestionPipeline,
    FilterCriteria,
    AnswerType,
    AnswerTypeEnum,
)
from lightningrod._generated.models import QuestionRenderer

api_key = os.getenv("LIGHTNINGROD_API_KEY", "your-api-key-here")

client = LightningRod(api_key=api_key)

## Multiple Choice Answer Type Overview

Multiple choice questions provide a question with several predefined answer options (typically 2-5 choices). The `QuestionAndLabelGenerator` generates both the question and the answer options together, then determines which option is correct.

Ideal for:
- Educational content and quizzes
- Knowledge testing
- Structured predictions with fixed choices
- Scenarios where you want to limit possible answers

In [ ]:
answer_type = AnswerType(answer_type=AnswerTypeEnum.MULTIPLE_CHOICE)

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    interval_duration_days=7,
    search_query="technology industry news",
)

## Configure QuestionAndLabelGenerator

For multiple choice questions, use `QuestionAndLabelGenerator` instead of separate `QuestionGenerator` and `WebSearchLabeler`. This generator creates both the question and answer options, then labels the correct choice.

In [ ]:
question_and_label_generator = QuestionAndLabelGenerator(
    instructions=(
        "Generate multiple choice questions about technology industry events. "
        "Each question should have 3-4 answer options, with one clearly correct answer."
    ),
    examples=[
        "Question: Which company will release a new AI model first in 2025?\n"
        "A) OpenAI\n"
        "B) Google\n"
        "C) Anthropic\n"
        "Label: A",
        "Question: What will be the main focus of Apple's next product launch?\n"
        "A) AI features\n"
        "B) Battery life\n"
        "C) Camera improvements\n"
        "D) Design changes\n"
        "Label: A",
    ],
    bad_examples=[
        "Question: Will AI be important? Label: Yes",
        "Question: What is AI? Label: Artificial Intelligence",
    ],
    filter_=FilterCriteria(
        rubric="The question must be a well-formed multiple choice question with clear answer options",
        min_score=0.7
    ),
    questions_per_seed=2,
    answer_type=answer_type,
)

In [ ]:
renderer = QuestionRenderer(answer_type=answer_type)

## Run the Pipeline

Note: For multiple choice questions, we don't use a separate `labeler` because `QuestionAndLabelGenerator` handles both question generation and labeling.

In [ ]:
pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_and_label_generator,
    renderer=renderer,
)

dataset = client.transforms.run(pipeline_config, max_questions=20)

## Interpreting Multiple Choice Labels

Multiple choice labels indicate which option (A, B, C, D, etc.) is the correct answer. The label contains the letter of the correct choice.

In [ ]:
samples = dataset.to_samples()

print(f"Generated {len(samples)} multiple choice questions\n")

for i, sample in enumerate(samples[:5]):
    print(f"Question {i+1}:")
    if sample.question:
        print(f"  {sample.question.question_text}")
    if sample.label:
        print(f"  Correct Answer: {sample.label.label}")
        print(f"  Confidence: {sample.label.label_confidence:.2f}")
    print()

## Use Cases for Multiple Choice Questions

**Educational Content:**
- Quiz questions for learning materials
- Knowledge assessment questions
- Study guides

**Structured Predictions:**
- "Which team will win?" with team options
- "What will be the outcome?" with predefined scenarios
- "Which product will launch first?" with product options